In [1]:
# simple but success

In [2]:
import numpy as np
import gym
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

In [3]:
env = gym.make('CartPole-v0')
obs = env.reset()
env.render()

In [4]:
env.action_space.sample()

0

In [5]:
training_data = []

trianing_size = 200
score_thresh = 50

times = 0
while len(training_data)<trianing_size:
    print("Trying %s..."%times)
    env = gym.make('CartPole-v0')
    obs = env.reset()
    done = False
    record = []
    score = 0
    while not done:
        action = env.action_space.sample()
        curr_obs, reward, done, _ = env.step(action)
        record.append([obs, action])
        obs = curr_obs
        score += reward
    times+=1
    if score>score_thresh:
        training_data.append(record)

Trying 0...
Trying 1...
Trying 2...
Trying 3...
Trying 4...
Trying 5...
Trying 6...
Trying 7...
Trying 8...
Trying 9...
Trying 10...
Trying 11...
Trying 12...
Trying 13...
Trying 14...
Trying 15...
Trying 16...
Trying 17...
Trying 18...
Trying 19...
Trying 20...
Trying 21...
Trying 22...
Trying 23...
Trying 24...
Trying 25...
Trying 26...
Trying 27...
Trying 28...
Trying 29...
Trying 30...
Trying 31...
Trying 32...
Trying 33...
Trying 34...
Trying 35...
Trying 36...
Trying 37...
Trying 38...
Trying 39...
Trying 40...
Trying 41...
Trying 42...
Trying 43...
Trying 44...
Trying 45...
Trying 46...
Trying 47...
Trying 48...
Trying 49...
Trying 50...
Trying 51...
Trying 52...
Trying 53...
Trying 54...
Trying 55...
Trying 56...
Trying 57...
Trying 58...
Trying 59...
Trying 60...
Trying 61...
Trying 62...
Trying 63...
Trying 64...
Trying 65...
Trying 66...
Trying 67...
Trying 68...
Trying 69...
Trying 70...
Trying 71...
Trying 72...
Trying 73...
Trying 74...
Trying 75...
Trying 76...
Trying 77

In [6]:
len(training_data)

200

In [7]:
X=[]
Y=[]
for data in training_data:
    for x, y in data:
        X.append(x)
        Y.append(y)
tensorX = torch.from_numpy(np.array(X)).float()
tensorY = torch.from_numpy(np.array(Y))

In [8]:
tensorX


 3.3494e-02  3.3806e-02  4.8608e-02  1.0521e-02
 3.4170e-02 -1.6198e-01  4.8818e-02  3.1814e-01
 3.0930e-02 -3.5776e-01  5.5181e-02  6.2581e-01
                       ⋮                        
 2.8907e-01  7.6793e-01 -1.6259e-01 -9.7013e-01
 3.0443e-01  9.6482e-01 -1.8199e-01 -1.3092e+00
 3.2372e-01  1.1617e+00 -2.0817e-01 -1.6528e+00
[torch.FloatTensor of size 12563x4]

In [9]:
trial_dataset = TensorDataset(tensorX, tensorY)

In [10]:
trial_dataset[3]

(
  0.0238
 -0.1635
  0.0677
  0.3510
 [torch.FloatTensor of size 4], 1)

In [11]:
model = nn.Sequential(nn.Linear(4,16), nn.ReLU(), 
                     nn.Linear(16,32), nn.ReLU(), 
                     nn.Linear(32,16), nn.ReLU(), 
                     nn.Linear(16,2))
optimizer = optim.RMSprop(model.parameters(),lr=1.e-3)
critic = nn.CrossEntropyLoss()

In [12]:
dataloader = DataLoader(trial_dataset, batch_size=64)

In [13]:
epochs = 50
for _ in range(epochs):
    for x, y in dataloader:
        vx = Variable(x,requires_grad=False)
        vy = Variable(y,requires_grad=False)
        pred = model(vx)
        loss = critic(pred,vy)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(loss.data[0])

0.6875895857810974
0.6852441430091858
0.696462094783783
0.6836995482444763
0.6948955655097961
0.6828756928443909
0.7146509885787964
0.676708996295929
0.6924504637718201
0.6774025559425354
0.6906996369361877
0.6845117211341858
0.6794983744621277
0.686005711555481
0.6726095676422119
0.6808896660804749
0.700379490852356
0.6880066394805908
0.6767441034317017
0.6701518297195435
0.6804279685020447
0.6884449124336243
0.690157949924469
0.6947409510612488
0.6715723276138306
0.6719685792922974
0.6728397011756897
0.668181836605072
0.6788888573646545
0.6858674883842468
0.673589289188385
0.6896411776542664
0.6683253049850464
0.6777289509773254
0.6911796927452087
0.6732049584388733
0.6689101457595825
0.6740633249282837
0.672605574131012
0.6903377771377563
0.6651839017868042
0.6839845180511475
0.675800085067749
0.6779195070266724
0.6587628126144409
0.6800146698951721
0.675363302230835
0.6667463779449463
0.7264297604560852
0.6916977167129517
0.6895682215690613
0.6767662763595581
0.7147835493087769
0.6

In [19]:
env = gym.make('CartPole-v1')
obs = env.reset()
done = False
score = 0
total_score = 0
countdown=100
while countdown>0:
    vo = Variable(torch.from_numpy(obs).float().unsqueeze(0), volatile=True)
    pred = model(vo)
    action = pred.max(dim=1)[1].data[0]
#     print(action)
    obs, reward, done, _ = env.step(action)
    total_score+= reward
    if done:
        countdown-=1
    else:
        score+=reward
    env.render()
print(score, total_score)

499.0 599.0
